In [1]:
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
import seaborn as sns
df=sns.load_dataset('tips')
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [2]:
X=df.iloc[:,1:]
y=df['total_bill']
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

In [3]:
numeric_preprocessor = Pipeline(
 steps=[
 ("imputation_mean", SimpleImputer(missing_values=np.nan,
strategy="mean")),
 ("scaler", StandardScaler()),
 ]
)

In [4]:
from sklearn import set_config
set_config(display='diagram')

In [5]:
numeric_preprocessor


Pipeline(steps=[('imputation_mean', SimpleImputer()),
                ('scaler', StandardScaler())])

In [6]:
categorical_preprocessor = Pipeline(
 steps=[
 (
 "imputation_constant",
 SimpleImputer(fill_value="missing", strategy="constant"),
 ),
 ("onehot", OneHotEncoder(handle_unknown="ignore")),
 ]
)

In [7]:
preprocessor=Pipeline(
 steps=[("categorical",categorical_preprocessor),
("numerical",numeric_preprocessor)]
)

In [8]:
preprocessor

Pipeline(steps=[('categorical',
                 Pipeline(steps=[('imputation_constant',
                                  SimpleImputer(fill_value='missing',
                                                strategy='constant')),
                                 ('onehot',
                                  OneHotEncoder(handle_unknown='ignore'))])),
                ('numerical',
                 Pipeline(steps=[('imputation_mean', SimpleImputer()),
                                 ('scaler', StandardScaler())]))])

In [9]:
pipe=Pipeline(
 [("preprocessor",preprocessor),
("regressor",RandomForestRegressor())]
)

In [10]:
pipe

Pipeline(steps=[('preprocessor',
                 Pipeline(steps=[('categorical',
                                  Pipeline(steps=[('imputation_constant',
                                                   SimpleImputer(fill_value='missing',
                                                                 strategy='constant')),
                                                  ('onehot',
                                                   OneHotEncoder(handle_unknown='ignore'))])),
                                 ('numerical',
                                  Pipeline(steps=[('imputation_mean',
                                                   SimpleImputer()),
                                                  ('scaler',
                                                   StandardScaler())]))])),
                ('regressor', RandomForestRegressor())])

In [11]:
from sklearn.compose import ColumnTransformer

In [12]:
categorical_cols = X.select_dtypes(include='object').columns.tolist()
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

In [13]:
preprocessor = ColumnTransformer(
    transformers=[
        ("categorical", categorical_preprocessor, categorical_cols),
        ("numerical", numeric_preprocessor, numerical_cols),
    ]
)

In [14]:
pipe = Pipeline(
    [("preprocessor", preprocessor), ("regressor", RandomForestRegressor())]
)